Copyright 2024 - Forusone : shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# Gemini Pro - Function call

* This notebook explains how to use function calling feature in Gemini.

* Refer to https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/function-calling

# Configuration
## Install python packages
* Vertex AI SDK for Python
  * https://cloud.google.com/python/docs/reference/aiplatform/latest


In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 38.8 MB/s eta 0:00:00


In [2]:
from IPython.display import display, Markdown

## Authentication to access to the GCP & Google drive

* Use OAuth to access the GCP environment.
 * Refer to the authentication methods in GCP : https://cloud.google.com/docs/authentication?hl=ko

In [3]:
#  For only colab to authenticate to get an access to the GCP.
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

    # To access contents in Google drive
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


# Execute the example
## Set the environment on GCP Project
* Configure project information
  * Model name : LLM model name : https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models
  * Project Id : prodect id in GCP
  * Region : region name in GCP

In [4]:
MODEL_NAME="gemini-1.5-flash"
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"

### Vertex AI initialization
Configure Vertex AI and access to the foundation model.
* Vertex AI initialization : aiplatform.init(..)
  * https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization

In [5]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part, Tool
import vertexai.generative_models as generative_models

# Grounding service is still in preview.
from vertexai.preview.generative_models import grounding

# Initalizate the current vertex AI execution environment.
vertexai.init(project=PROJECT_ID, location=REGION)

# Access to the generative model.
model = GenerativeModel(MODEL_NAME)

## Function to be called by LLM
* The following function will be called by LLM if the condition is satisfied.

In [6]:
def get_stock_price(ticker:str) ->str:
  """
  The ticker passed to Arg is the value found by LLM.
  If you say, Tell me the price of Google stock, the argument passed to arg here can be GOOG or GOOGL.

  ticker : str
  Stock ticker

  return : str
  Stock price
  """

  print(f"Request ticker : {ticker}")

  return "Stock price is 445.55"


## Function including Gemini function calling logic.

In [7]:
from vertexai.generative_models import FunctionDeclaration
from vertexai.generative_models import GenerativeModel, Part, Tool

def function_call(prompt:str)->str:

  # Tools
  tools = Tool(function_declarations=[
      FunctionDeclaration(
          name="get_stock_price",
          description="Get the current stock price of a given company",
          parameters={
              "type": "object",
              "properties": {
                  "ticker": {
                      "type": "string",
                      "description": "Stock ticker symbol"
                  }
              }
          },
      )
  ])

  # Model Initialization
  model = GenerativeModel("gemini-pro",
                          generation_config={"temperature": 0},
                          tools=[tools])

  chat = model.start_chat()

  # Send a prompt to the chat
  response = chat.send_message(prompt)
  function_call = response.candidates[0].content.parts[0].function_call

  function_handlers = {
      "get_stock_price": get_stock_price,
  }

  if function_call.name in function_handlers:
      function_name = function_call.name
      chat_response = ""

      args = {key: value for key, value in function_call.args.items()}

      if args:
          function_response = function_handlers[function_name](args)

          part_data = Part.from_function_response(
                  name=function_name,
                  response={
                      "content": function_response,
                  }
          )

          cal_eq = part_data.to_dict()['function_response']['response']['content']

          response = chat.send_message(part_data,)
          chat_response = response.candidates[0].content.parts[0].text

          print(f"Response with function calling: {chat_response}")
          return chat_response
      else:
          print("No arguments found for the function.")
          return "Please let me know the name of company that you want to know the stock price"
  else:
      print("Response without function calling")
      return response.text


## Execute the example

### Response with a prompt that becomes a function call
* In this case, if the LLM determines a prompt that a currently registered function needs to be called when considering contents, the function could be called, the LLM receives the result, creates the corresponding value, and returns it.

In [8]:
prompt = "How much is the Google's stock price?"
function_call(prompt)


Request ticker : {'ticker': 'GOOG'}
Response with function calling: The current stock price of Google (GOOG) is 445.55.


'The current stock price of Google (GOOG) is 445.55.'

### Reponse with a prompt where the function call is not called
* In this case, the LLM determines the contents of the prompt and there is no function to be called.

In [9]:
prompt = "How is the weather in Seoul?"
function_call(prompt)


Response without function calling


'I am sorry, I cannot fulfill this request. The available tools lack the desired functionality.'